In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


# Fake News Detection using BERT

This project aims to classify news articles as **real or fake** using a fine-tuned BERT (Bidirectional Encoder Representations from Transformers) model. With the increasing spread of misinformation, automating fake news detection has become crucial. 

We leverage the power of **transformer-based models** to understand the context and semantics of the news text and make binary predictions.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [ ]:
df_fake.head()

In [ ]:
df_true.head()

In [ ]:
df_fake["label"] = 0
df_true["label"] = 1

In [ ]:
df_fake.shape, df_true.shape

In [ ]:
df = pd.concat([df_fake, df_true]).reset_index(drop=True)

In [ ]:
df = df[["title", "label"]].dropna()

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.sample(frac = 1)
df.head()

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [ ]:
print(df.columns)

In [ ]:
df["title"] = df["title"].apply(wordopt)

In [ ]:
print(df.columns)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot class distribution
sns.countplot(data=df, x="label")
plt.title("Distribution of Fake (0) and Real (1) News")
plt.xlabel("Label")
plt.ylabel("Count")
plt.xticks([0, 1], ['Fake', 'Real'])
plt.show()

# Count values
print(df["label"].value_counts())

In [ ]:
df["text_length"] = df["title"].apply(lambda x: len(x.split()))

plt.figure(figsize=(10, 5))
sns.histplot(data=df, x="text_length", bins=30, hue="label", kde=True, palette="Set2")
plt.title("Title Word Count Distribution by Class")
plt.xlabel("Number of Words")
plt.ylabel("Frequency")
plt.legend(labels=["Real", "Fake"])
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud

fake_titles = df[df['label'] == 0]['title']
real_titles = df[df['label'] == 1]['title']

# Create wordclouds
fake_text = " ".join(fake_titles)
real_text = " ".join(real_titles)

plt.figure(figsize=(16, 6))

# Fake news word cloud
plt.subplot(1, 2, 1)
wordcloud_fake = WordCloud(width=800, height=400, background_color='white').generate(fake_text)
plt.imshow(wordcloud_fake, interpolation='bilinear')
plt.title("WordCloud for Fake News")
plt.axis('off')

# Real news word cloud
plt.subplot(1, 2, 2)
wordcloud_real = WordCloud(width=800, height=400, background_color='white').generate(real_text)
plt.imshow(wordcloud_real, interpolation='bilinear')
plt.title("WordCloud for Real News")
plt.axis('off')

plt.show()


In [ ]:
plt.figure(figsize=(5, 5))
df['label'].value_counts().plot.pie(autopct='%1.1f%%', labels=['Fake', 'Real'], colors=['red', 'green'])
plt.title("Fake vs Real News")
plt.show()

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df["title"], df["label"], test_size=0.2, random_state=42)


In [ ]:
# Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

In [ ]:
print(df.head())


In [ ]:
# Dataset conversion
class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [ ]:
# Load pre-trained model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
# Define metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
from datasets import Dataset
import torch

# Combine tokenized inputs and labels
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",  # ← adjusted earlier
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,     
    eval_dataset=val_dataset,       
    compute_metrics=compute_metrics
)


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


In [ ]:
trainer.save_model("saved_model")
tokenizer.save_pretrained("saved_model")
print("Model and tokenizer saved successfully.")

# 📌 Conclusion

The BERT-based fake news detection model achieved good performance on the validation set using just 2 epochs. With metrics like precision, recall, and F1-score computed, we see that BERT can understand and differentiate between fake and real news effectively.

### Key Takeaways:
- BERT effectively captures contextual nuances in text classification.
- The model performed well even with limited epochs and preprocessing.
- Fine-tuning pre-trained models is a powerful technique for NLP tasks.

### ✅ Future Scope:
- Train for more epochs or fine-tune hyperparameters.
- Use a larger and more diverse dataset.
- Deploy the model via a web app or REST API.
